# Data loader experiments

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data import *
from local.notebook.showdoc import show_doc

In [ ]:
from torch.utils.data import IterableDataset, get_worker_info

In [ ]:
import string
letters = list(string.ascii_lowercase)

In [ ]:
bs = 4

In [ ]:
def twoepochs(d): print(' '.join(''.join(o) for _ in range(2) for o in d))

In [ ]:
@patch
def __len__(self:DataLoader):
    return len(self.dataset) if isinstance(self.dataset, IterableDataset) else len(self.index_sampler)

## patch \_MapDatasetFetcher 

In [ ]:
@patch
def fetch(self:torch.utils.data._utils.fetch._MapDatasetFetcher, idxs):
    ds = self.dataset
    if self.auto_collation:
        if hasattr(self.collate_fn,"get_batch"): return self.collate_fn.get_batch(ds, idxs)
        data = [ds[idx] for idx in idxs]
    else: data = ds[idxs]
    return self.collate_fn(data)

In [ ]:
class CustCollate(CollateFunc):
    def get_batch(self, ds, idxs): return self([ds[idx] for idx in idxs])

In [ ]:
ds = string.ascii_lowercase
dl = DataLoader(ds, batch_size=4, num_workers=0, collate_fn=CustCollate())

In [ ]:
[''.join(o) for o in dl]

['abcd', 'efgh', 'ijkl', 'mnop', 'qrst', 'uvwx', 'yz']

## BatchDS

In [ ]:
class SleepyDS():
    def __init__(self,coll): self.coll=coll
    def __len__(self): return len(self.coll)
    def __getitem__(self,i): time.sleep(0.02); return self.coll[i]

In [ ]:
class BatchDS(IterableDataset):
    def __init__(self, ds ,bs=1, shuffle=False, sampler=None, batch_sampler=None, drop_last=False,
                sampler_cls=None, batch_sampler_cls=BatchSampler):
        self.ds,self.samp,self.rng,self.nw,self.offs = ds,batch_sampler,random.Random(),1,0
        if self.samp: return
        if not sampler:
            if not sampler_cls: sampler_cls = RandomSampler if shuffle else SequentialSampler
            sampler = sampler_cls(ds)
        self.samp = batch_sampler_cls(sampler, bs, drop_last)
        
    def __len__(self): return len(self.samp)
        
    def __iter__(self):
        return ([self.ds[j] for j in b] for i,b in enumerate(self.samp) if i%self.nw==self.offs)

In [ ]:
def _wif(worker_id):
    info = get_worker_info()
    ds = info.dataset
    ds.nw,ds.offs = info.num_workers,info.id
    torch.manual_seed(ds.rng.randint(0,sys.maxsize))
    ds.samp.sampler = copy.copy(ds.samp.sampler)

def get_dl(bs=4, collate_fn=default_collate, num_workers=0, **kwargs):
    dsi = BatchDS(SleepyDS(string.ascii_lowercase), bs=4, **kwargs)
    return DataLoader(dsi, num_workers=num_workers, batch_size=None, worker_init_fn=_wif, collate_fn=collate_fn)

In [ ]:
dl = get_dl(bs=4, num_workers=4, shuffle=True)
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

jkuc prsv aweo fmtl zhdg qnby ix jkuc prsv aweo fmtl zhdg qnby ix


In [ ]:
dl = get_dl(bs=4, num_workers=4)
%time twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz
CPU times: user 18.3 ms, sys: 43.8 ms, total: 62.1 ms
Wall time: 390 ms


In [ ]:
dl = get_dl(bs=4, num_workers=0)
%time twoepochs(dl)
len(dl)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz
CPU times: user 1.83 ms, sys: 733 µs, total: 2.56 ms
Wall time: 1.05 s


7

In [ ]:
dl = get_dl(bs=4, num_workers=4, drop_last=True)
twoepochs(dl)
len(dl)

abcd efgh ijkl mnop qrst uvwx abcd efgh ijkl mnop qrst uvwx


6

In [ ]:
dl = get_dl(bs=4, num_workers=0, shuffle=True)
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

vjgu zhdm owsn kxyl icqb frte pa uzyw rgak vqdl txpe hsfb icjm no


In [ ]:
ds = SleepyDS(string.ascii_lowercase)
dl = get_dl(bs=4, num_workers=4, sampler=SequentialSampler(ds))
twoepochs(dl)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz


In [ ]:
dl = get_dl(num_workers=4, batch_sampler=BatchSampler(RandomSampler(ds), 8, False))
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

adcpyuoe fhtwgnql mvxbksij zr adcpyuoe fhtwgnql mvxbksij zr


In [ ]:
def rev_collate(s): return default_collate(list(reversed(s)))
dl = get_dl(bs=4, num_workers=4, collate_fn=rev_collate)
twoepochs(dl)

dcba hgfe lkji ponm tsrq xwvu zy dcba hgfe lkji ponm tsrq xwvu zy
